Authors: Negin Sobhani, Jakidxav

This Notebook is slightly different than the `ghcn_Import` Notebook, in that it creates hot and not-hot labels for the entire Eastern United States instead of one single station. We still subset for the right years and calendar days, and still compute an anomaly column for the maximum temperature for each station. However, instead of comparing one temperature to the cutoff value, we will compare whether all stations had greater than or equal to the mean + 1 standard deviation temperature anomalies across days and stations. In this way, we can create labels that account for every station in the Eastern US at the same time.

As in the last Notebook, we can still create the training, development, and validation set labels at the same time.

In [0]:
import netCDF4 as nc
import pickle

import numpy as np
import datetime as dt
import pandas as pd

import matplotlib.pylab as plt
%matplotlib inline

In [0]:
ghcnd_csv_dir = '/glade/p/work/ddvento/ML/McKinnon_data/ghcnd/ghcnd_all_csv/'
        
#starting and end years for loading in data
start_year = 1982
end_year   = 2015

#the days we care about; specified in mckinnon's paper
start_doy  = 175
end_doy    = 234
  
#cutoff for choosing whether the day was anomalously hot or not
cut_off = 6.5
count = 0

#dev/test set include 2 el nino years, 1 non-el nino years
#https://www.esrl.noaa.gov/psd/enso/past_events.html
dev_nino_list = [1983, 1990, 1995, 2008]
val_nino_list = [1988, 1994, 1999, 2003]

In [0]:
'''
get_ghcnd_stn:
    *Opens the csv file for the GHCND station as df
    *Add a pd.datetime column to the df
    *Add Julian Day (day of the year) jday to the df 
    *Selects data for only the training years
    *Selects data for only the selected days of a year ( e.g. 60 days of summer.) 

----------
Parameters:
    ghcnd_csv_dir --- path to GHCN processed csv files.
    stn_id --- GHCN station ID based on GHCN readme.txt
        (ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt)

    start_year --- start year for the period to slice. 
    end_year 

    start_doy  --- start julian day (day of year) for the period
                    you are looking at.  
    end_doy
-------
Returns:
    stn_data --- 

-------
Example:
    stn_data = get_ghcnd_stn(ghcnd_csv_dir,stn_id,1982,
                2015, 30, 90)

'''
def get_ghcnd_stn (ghcnd_csv_dir, stn_id, start_year, end_year, start_doy, end_doy):
        #create station label so that we can read in the file using pandas
        stn_csv = ghcnd_csv_dir+stn_id+'.csv'
        
        #read in file, replace nan values
        stn_raw = pd.read_csv(stn_csv,na_values=-9999)
            
        #convert dates into datetime objects so that we can extract the day of the year
        stn_raw['date']=pd.to_datetime(stn_raw['YYYY'].astype(str)+'-'+stn_raw['MM'].astype(str)+'-'+stn_raw['DD'].astype(str))
        stn_raw['jday'] = stn_raw['date'].dt.dayofyear.apply(lambda x: str(x).zfill(3)).astype(int)

        #subset data based on years and calendar days
        yrs_data = stn_raw[(stn_raw['YYYY']>=start_year) & (stn_raw['YYYY']<=end_year)]
        stn_data= yrs_data[(yrs_data['jday']>=start_doy) & (yrs_data['jday']<=end_doy)]
            
        return stn_data

In [0]:
'''
calc_stn_anom :
    *Calculates the anomalies of selected var for the station.
----------
Parameters:
    stn_data ---
    var --- Name of the varibale to calculate anomalies on :
            e.g. TMAX, TMIN, PRCP
-------
Returns:
    stn_data --- 

-------
Example:
    calc_stn_anom (stn_data, 'TMAX')
    '''
def calc_stn_anom (stn_data, var):
        #create an anomaly column variable in our dataframe
        var_anom= var+"ANOM"
        
        #create mean of a given variable by the .groupby() method and applying a mean transform
        means=stn_data.groupby(['MM','DD'])[var].transform('mean')
            
        #create anomaly by subtracting mean
        stn_data[var_anom]= stn_data[var] - means
            
        return stn_data

In [0]:
#picked this station because I know it has full 2040 values
#thus multi-index will have correct dimensions for concatenation
station_data = get_ghcnd_stn(ghcnd_csv_dir, 'USC00391621', start_year, end_year, start_doy, end_doy)

In [0]:
#calculate the station anomaly for the maximum temperature variable
station_data = calc_stn_anom(station_data, 'TMAX')

#drop every column except for the anomaly column
station_data.drop(['MM', 'DD', 'PRCP', 'SNOW', 'SNWD', 'date', 'TMIN', 'TMAX'], axis=1, inplace=True)

#set the index to be the year and the calendar day
station_data.set_index(['YYYY','jday'], inplace=True)
station_data.head()

TMAXANOM
YYYY jday          
1982 175   2.938235
     176  -8.567647
     177  -6.014706
     178  -1.920588
     179   2.338235

In [0]:
#here we can load in another station, and complete the same steps as we did for `station_data`
station_data2 = get_ghcnd_stn(ghcnd_csv_dir, 'USC00393832', start_year, end_year, start_doy, end_doy)
station_data2 = calc_stn_anom(station_data2, 'TMAX')
station_data2.drop(['MM', 'DD', 'PRCP', 'SNOW', 'SNWD', 'date', 'TMIN', 'TMAX'], axis=1, inplace=True)
station_data2.set_index(['YYYY','jday'], inplace=True)

#then we can concatenate the two stations together
big_station = pd.concat([station_data, station_data2], axis=1)

In [0]:
big_station.head()

TMAXANOM  TMAXANOM
YYYY jday                    
1982 175   2.938235       NaN
     176  -8.567647 -6.335484
     177  -6.014706 -1.025806
     178  -1.920588  2.165625
     179   2.338235  5.320690

Okay! That worked. So now we can load in all of the other stations in our directory and perform the same steps. We are doing this to create a single label for every station in our data set. Thus, we will be predicting whether the entire Eastern US is anomalously hot instead of one specific station.

McKinnon also does this in her paper. Because of the variablility in prediction skill between stations, we can create a new label to represent all of them.

In [0]:
for file in os.listdir(ghcnd_csv_dir):
    if(np.logical_and(file != 'USC00391621.csv', file != 'USC00393832.csv')):
        count = count + 1
        
        stn_id = file.replace('.csv', '')
        stn_data = get_ghcnd_stn(ghcnd_csv_dir, stn_id, start_year, end_year, start_doy, end_doy)
        stn_data = calc_stn_anom(stn_data, 'TMAX')
        stn_data.drop(['MM', 'DD', 'PRCP', 'SNOW', 'SNWD', 'date', 'TMIN', 'TMAX'], axis=1, inplace=True)
        stn_data.set_index(['YYYY','jday'], inplace=True)
        
        big_station = pd.concat([big_station, stn_data], axis=1)

In [0]:
#percentile across rows in dataframe
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.quantile.html
big_station['quant'] = big_station.quantile(0.95, axis=1)

In [0]:
print(big_station.quant)

YYYY  jday
1982  175     2.144848
      176     1.551925
      177     1.296279
      178     2.184848
      179     3.604355
      180     3.942647
      181     2.692197
      182     1.912993
      183     2.234941
      184     4.854266
      185     6.813182
      186     7.101324
      187     5.225680
      188     4.017458
      189     4.314706
      190     3.437647
      191     3.020588
      192     3.142103
      193     2.957059
      194     3.234820
      195     3.120321
      196     3.372948
      197     4.529661
      198     4.708824
      199     4.607353
      200     4.626471
      201     5.269091
      202     4.824492
      203     3.794875
      204     3.625865
                ...   
2015  205     3.807241
      206     4.186471
      207     3.683323
      208     4.261765
      209     4.786765
      210     5.146257
      211     5.015931
      212     3.776879
      213     3.161858
      214     3.413002
      215     3.752121
      216     3.387419


In [0]:
#calculate mean and standard deviation of the 95th percentile
mean = big_station.quant.mean()
stdev = big_station.quant.std()

print(mean, stdev)

4.262958844169827 2.0954880217628777


In [0]:
#create a second label that is greater than the mean + 1 standard deviation in temperatures
big_station['HOT2'] = np.where(big_station.quant >= mean+stdev, 1, 0)

print(np.count_nonzero(big_station.HOT2))

In [0]:
#reset our dataframe's index
big_station.reset_index(inplace=True)

#then create the development set like we did before
#dev_nino_list = [1983, 1990, 1995, 2008]
_1983 = big_station[big_station['YYYY'] == 1983]
_1990 = big_station[big_station['YYYY'] == 1990]
_1995 = big_station[big_station['YYYY'] == 1995]
_2008 = big_station[big_station['YYYY'] == 2008]

hot_dev = pd.concat([_1983, _1990, _1995, _2008])

Y_dev = hot_dev.HOT2.values

In [0]:
print(Y_dev.shape)

(240,)


In [0]:
#same for the validation set
#val_nino_list = [1988, 1994, 1999, 2003]
_1988 = big_station[big_station.YYYY == 1988]
_1994 = big_station[big_station.YYYY == 1994]
_1999 = big_station[big_station.YYYY == 1999]
_2003 = big_station[big_station.YYYY == 2003]

hot_val = pd.concat([_1988, _1994, _1999, _2003])

Y_val = hot_val.HOT2.values

In [0]:
print(Y_val.shape)

(240,)


In [0]:
#and lastly our training set
#for every year not in our dev or val sets, add it to the training set
list_years = [1983, 1988, 1990, 1994, 1995, 1999, 2003, 2008]

for i in list_years:
    big_station = big_station[big_station.YYYY != i]
    
Y_train = big_station.HOT2.values

In [0]:
#make directories to save to if they aren't already there
os.mkdir('/glade/work/jakidxav/IPython/20_lead/Y_train/station0/')
os.mkdir('/glade/work/jakidxav/IPython/20_lead/Y_dev/station0/')
os.mkdir('/glade/work/jakidxav/IPython/20_lead/Y_val/station0/')

#then pickle the labels
Y_train_filename = '/glade/work/jakidxav/IPython/20_lead/Y_train/station0/Y_train.txt'
with open(Y_train_filename, 'wb') as f:
    pickle.dump(Y_train, f)
    
Y_dev_filename = '/glade/work/jakidxav/IPython/20_lead/Y_dev/station0/Y_dev.txt'
with open(Y_dev_filename, 'wb') as g:
    pickle.dump(Y_dev, g)
    
Y_val_filename = '/glade/work/jakidxav/IPython/20_lead/Y_val/station0/Y_val.txt'
with open(Y_val_filename, 'wb') as h:
    pickle.dump(Y_val, h)